# 1. Download Portfolio Data
## MMAI823 - AI in Finance
## Team Humphrey - December 7, 2019

This notebook downloads all necessary data for the "2. Run Portfolio Optimization notebook"

In [11]:
import yfinance as yf  # conda install -c ranaroussi yfinance
import numpy as np
import pandas as pd 

import bs4 as bs
import requests
import pickle

In [12]:
# Reference: https://pythonprogramming.net/sp500-company-list-python-programming-for-finance/
def download_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    tables = soup.find_all('table', {'class': 'wikitable sortable'})
    
    # First table - SP500 composition
    table = tables[0]
    table_rows = table.find_all('tr')
    
    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td]
        l.append(row)
    df_tickers = pd.DataFrame(l)
    
    df_tickers.drop([0], axis=0, inplace=True)
    df_tickers.columns = ['Ticker', 'Security', 'SEC filings', 'GICS Sector', 'GICS Sub Industry', 
                          'Headquarters Location', 'Date first added', 'CIK', 'Founded']
    
    # Second table - SP500 stocks added and removed
    table = tables[1]
    table_rows = table.find_all('tr')
    
    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td]
        l.append(row)
    df_addremoved = pd.DataFrame(l)
    
    df_addremoved.drop([0, 1], axis=0, inplace=True)
    df_addremoved.columns = ['Date', 'Ticker Added', 'Security Added', 'Ticker Removed', 'Security Removed', 'Comment']
    
    return df_tickers, df_addremoved

In [13]:
# Download SP500 tickers composition
df_tickers, df_addremoved = download_sp500_tickers()

# Save to Excel
with pd.ExcelWriter('SP500 Info.xlsx') as writer: 
    df_tickers.to_excel(writer, sheet_name='SP500_Tickers', index=False)
    df_addremoved.to_excel(writer, sheet_name='SP500_AddRem', index=False)

# Pickle
df_tickers.to_pickle("df_tickers.pickle")
df_addremoved.to_pickle("df_addremoved.pickle")

In [15]:
df_addremoved

,Date,Ticker Added,Security Added,Ticker Removed,Security Removed,Comment
2,"November 21, 2019",NOW,ServiceNow,CELG,Celgene,Bristol-Myers Squibb Co. (NYSE:BMY) acquired C...
3,"October 3, 2019",LVS,Las Vegas Sands,NKTR,Nektar Therapeutics,Market Cap changes[8]
4,"September 26, 2019",NVR,NVR Inc,JEF,Jefferies Financial Group Inc,JEF spinning off SPB[9]
5,"September 23, 2019",CDW,CDW,TSS,TSYS,S&P 500 constituent Global Payments Inc. (NYSE...
6,"August 9, 2019",LDOS,Leidos Holdings,APC,Anadarko Petroleum,S&P 500 & 100 constituent Occidental Petroleum...
...,...,...,...,...,...,...
216,"September 25, 2003",ESRX,Express Scripts,QTRN,Quintiles Transnational,Taken private[184]
217,"December 5, 2000",INTU,Intuit,BS,Bethlehem Steel,Market Cap changes.[185]
218,"December 5, 2000",SBL,Symbol Technologies,OI,Owens-Illinois,Market Cap changes.
219,"December 5, 2000",AYE,Allegheny Energy,GRA,W.R. Grace,Market Cap changes.


In [4]:
# Download price data
ticker_list = df_tickers['Ticker'].to_list()
ticker_string = ' '.join(ticker_list)

In [16]:
ticker_string

'MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CDW CE CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HRB HAL HBI HOG HIG HAS HCA PEAK HP HSIC HSY HES HPE HLT HFC HOLX HD HON 

In [17]:
df_ticker_data = yf.download(ticker_string, start="2014-11-28", end="2019-12-03")

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [18]:
df_ticker_data

Adj Close                                                \
                    A        AAL         AAP        AAPL       ABBV   
Date                                                                  
2014-11-28  40.715824  46.152267  145.852615  109.274948  56.694954   
2014-12-01  39.620270  45.534115  145.842728  105.728317  56.621220   
2014-12-02  39.991817  45.515099  147.240967  105.324020  56.752300   
2014-12-03  40.229969  45.876484  150.771225  106.518494  56.121452   
2014-12-04  40.268078  47.217396  151.971130  106.114204  56.998112   
...               ...        ...         ...         ...        ...   
2019-11-26  80.949997  29.049999  157.089996  264.290009  87.720001   
2019-11-27  81.080002  28.950001  158.779999  267.839996  88.330002   
2019-11-29  80.769997  28.740000  157.080002  267.250000  87.730003   
2019-12-02  80.349998  28.080000  156.550003  264.160004  87.029999   
2019-12-03  80.099998  27.290001  152.610001  259.450012  85.949997   

                                                                      ...  \
                  ABC        ABMD        ABT         ACN        ADBE  ...   
Date                                                                  ...   
2014-11-28  83.841881   35.520000  40.137791   77.905678   73.680000  ...   
2014-12-01  83.648506   35.000000  39.966457   77.607857   73.750000  ...   
2014-12-02  84.753487   35.279999  40.778053   77.589836   73.470001  ...   
2014-12-03  84.707458   35.430000  41.192871   78.077118   73.180000  ...   
2014-12-04  84.670624   35.650002  41.021538   77.914673   73.029999  ...   
...               ...         ...        ...         ...         ...  ...   
2019-11-26  87.849998  194.259995  85.419998  200.919998  307.899994  ...   
2019-11-27  88.510002  196.789993  85.419998  201.250000  309.059998  ...   
2019-11-29  87.910004  196.179993  85.449997  201.160004  309.529999  ...   
2019-12-02  87.830002  192.220001  84.510002  199.589996  302.750000  ...   
2019-12-03  87.430000  191.149994  84.360001  198.750000  303.910004  ...   

               Volume                                                          \
                  XEL       XLNX         XOM       XRAY        XRX        XYL   
Date                                                                            
2014-11-28  1210100.0  1423100.0  19556700.0   316000.0  1174200.0   676400.0   
2014-12-01  5969000.0  2638100.0  27584200.0  1170000.0  1633000.0   850000.0   
2014-12-02  3752000.0  1774100.0  20885300.0   616200.0  1916000.0   964300.0   
2014-12-03  3324100.0  5606100.0  16220400.0   681100.0  4057200.0   691700.0   
2014-12-04  2614200.0  4042600.0  12868600.0   915800.0  2621700.0   443000.0   
...               ...        ...         ...        ...        ...        ...   
2019-11-26  5882100.0  3923000.0  15457000.0  2583000.0  2688000.0  2154500.0   
2019-11-27  2685600.0  2026700.0   8393900.0   682200.0  1301100.0   794700.0   
2019-11-29  1730700.0  1362700.0   7980800.0   673900.0  1137800.0   624200.0   
2019-12-02  2623000.0  4049000.0  11116900.0  1877900.0  1421400.0   920100.0   
2019-12-03  2799400.0  4478900.0  12245800.0  1339200.0  3213300.0   800000.0   

                                                        
                  YUM        ZBH       ZION        ZTS  
Date                                                    
2014-11-28  2094000.0   426900.0  1766200.0  1332200.0  
2014-12-01  4344700.0   613000.0  3945000.0  3153000.0  
2014-12-02  5055100.0  1504900.0  1770900.0  2500600.0  
2014-12-03  4193800.0   975700.0  1698600.0  2003600.0  
2014-12-04  3092600.0   731900.0  1296700.0  4294300.0  
...               ...        ...        ...        ...  
2019-11-26  4723400.0   876600.0  1238900.0  3688700.0  
2019-11-27  2316300.0   487400.0  1097500.0  1443300.0  
2019-11-29   980600.0   351600.0   520600.0  1064400.0  
2019-12-02  1412900.0   509400.0  1243700.0  2050400.0  
2019-12-03  1914500.0   520500.0  1416100.0  2115200.0  

[126

In [ ]:
# Save to Excel
df_ticker_data.to_excel('SP500 Price Vol Data.xlsx', sheet_name = 'PriceVolHist')

# Pickle
df_ticker_data.to_pickle("df_ticker_data.pickle")

In [7]:
# Download all Beta's - this takes some time
ticker_data = []
for ticker in ticker_list:
    try:
        yf_t = yf.Ticker(ticker)
        ticker_data.append([ticker, 
                            yf_t.info['shortName'], 
                            yf_t.info['longName'], 
                            yf_t.info['beta'], 
                            yf_t.info['marketCap'],
                           ])
    except:
        ticker_data.append([ticker, 
                            '**error downloading**', 
                            '',
                            '',
                            '',
                           ])
                        
    df_ticker_info = pd.DataFrame(ticker_data, columns=['Ticker', 'ShortName', 'LongName', 'Beta', 'MarketCap'])  

In [19]:
yf_t = yf.Ticker('MSFT')

In [21]:
yf_t.info

{'zip': '98052',
 'sector': 'Technology',
 'fullTimeEmployees': 144000,
 'longBusinessSummary': "Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); and Skype, Outlook.com, and OneDrive. It also provides LinkedIn that includes Talent and marketing solutions, and subscriptions; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. The company's Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also provides support services and Microsoft 

In [10]:
# Save to Excel
df_ticker_info.to_excel('SP500 Beta MarketCap.xlsx', sheet_name = 'BetaMarketCap')

# Pickle
df_ticker_info.to_pickle("df_ticker_info.pickle")